In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import networkx as nx
from pybel.examples import sialic_acid_graph as sag
import pybel as pb
import json
import time
import csv
from pybel.io.jupyter import to_jupyter
import torch
import pyro
import pandas as pd
import numpy as np
pyro.set_rng_seed(101)

In [2]:
import covid19kg

In [3]:
import covid19kg
graph = covid19kg.get_graph()
graph.summarize()

Covid19KG v0.0.1-dev
Number of Nodes: 3954
Number of Edges: 9484
Number of Citations: 185
Number of Authors: 950
Network Density: 6.07E-04
Number of Components: 29
Number of Warnings: 0


In [4]:
dir(covid19kg)

['AUTHORS',
 'BELMetadata',
 'BELRepository',
 'HERE',
 'VERSION',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'get_graph',
 'get_graphs',
 'get_summary_df',
 'main',
 'metadata',
 'os',
 'repository',
 'serialize_authors']

In [5]:
#f = open('Epithelial Innate Immune Activation-2.0-Hs.jgf')
f = open('COVID19.jgf')
bel_statements = json.load(f)

In [8]:
# type has the structure of (type_parent, type_children, type_relation):[[edges]]
def get_information(jgf_file):
    # causal_relations = ['increases', 'decreases', 'directlyIncreases', 'directlyDecreases']
    types = []
    # nodes = set()
    parents = []
    parent_types = []
    children_types = []
    children = []
    str_list = []
    df = pd.DataFrame()
    
    for edges in jgf_file['graph']['edges']:
        parent = edges['source']
        child = edges['target']
        relation = edges['relation']
        text = edges['label']
        if relation.find('crease') > 0:
            # nodes.add(parent)
            # nodes.add(child)
            parents.append(parent)
            children.append(child)
            str_list.append(parent + '*' + relation + '*' + child)   
            parent_type = parent[:parent.find('(')]
            children_type = child[:child.find('(')]
            types.append(relation)
            children_types.append(children_type)
            parent_types.append(parent_type)
            # types.setdefault((parent_type,chidren_type,relation), []).append(parent + '*' + relation + '*' + child)
    df['parents'] = parents
    df['children'] = children
    df['types'] = types
    df['statements'] = str_list
    df['parent_types'] = parent_types
    df['children_types'] = children_types
    return df

In [9]:
df = get_information(bel_statements)

In [11]:
df.head()

,parents,children,types,statements,parent_types,children_types
0,path(MESH:COVID-19),path(MESH:Hypoalbuminemia),increases,path(MESH:COVID-19)*increases*path(MESH:Hypoal...,path,path
1,p(HGNC:JAK3),path(MESH:Lymphopenia),decreases,p(HGNC:JAK3)*decreases*path(MESH:Lymphopenia),p,path
2,"path(SDIS:""Liver Injury"")",p(HGNC:CRP),increases,"path(SDIS:""Liver Injury"")*increases*p(HGNC:CRP)",path,p
3,"bp(GO:""inflammatory response"")",p(HGNC:CRP),increases,"bp(GO:""inflammatory response"")*increases*p(HGN...",bp,p
4,a(SCHEM:fibrate),p(HGNC:CRP),decreases,a(SCHEM:fibrate)*decreases*p(HGNC:CRP),a,p


## To do:
1. type is causal relationship
2. we need to group parent and children type according to CBN legend
3. Start building SCM using the prior distribution guide Jeremy gave us
4. Train a linear model to confirm if the threshold / distribution works on the sample generated from SCM
5. Automate this process and test it on different BEL models we have

## Future scope
1. Counterfactuals and interventions